In [1]:
import pandas as pd

# Läs in maulerat-listan som jag döpt till Återbet.xlsx och sparat ner C:\Users\seosob02\

data = pd.read_excel("Återbet.xlsx")

# Kollar i kolumnen data["OrderNo"] och tar bort alla rader som inte är integers och större än 0

data = data[pd.to_numeric(data['OrderNo'], errors='coerce').notnull()]

# Se till att kolumnen data["OrderNo"] och data["PriceIncVat"] är ints och inte floats

data["OrderNo"] = data["OrderNo"].astype(int).replace('\.o', '', regex = True)

data["PriceIncVat"] = data["PriceIncVat"].astype(int).replace('\.o', '', regex = True)

# Använd bara kolumnerna som behövs för vidare kontering och spara dem i variabeln data

data = data[["ProductName", "OrderNo", "PriceIncVat", "PriceExcVat", "Info"]]

# Skapa en ny kolumn med momsen där man tar InkMoms - ExMoms

data["Moms"] = data["PriceIncVat"]-data["PriceExcVat"]

# Ta bort missing values (Nan) från kolumnen data["ProductName"] 

data.dropna(subset=['ProductName'], inplace = True)

# Skape ny kolumn som heter data["Konto"] som har 0-värde

data["Konto"] = None

# Kopiera datasetet med funktionen copy, så att vi kan använda samma dataset för att skapa Klarna-konteringen

data2 = data.copy()

# Skapa en ny kolumn som heter data2["Belopp"] med utgångsvärdet None

data2["Belopp"] = None

# Skapa ett nytt värde för kolumnen data2["PriceIncVat"]

data2["PriceIncVat"] = data2["PriceIncVat"]*-1

# Eftersom ena delen av datasetet ska bokföras mot konto 1570 så sätter vi data2["Konto"]= "1570"

data2["Konto"] = "1570"

# Skapa sedan fyra kolumner med inget värde, de representerar kst/kbär/projekt/artikel i konteringen

data2["B"] = None
data2["C"]= None
data2["D"] = None
data2["E"] = None

# Skapa strukturen för data2 så att den går att läsa in som en CSV-fil

data2 = data2[["Konto", "B", "C", "D", "E", "PriceIncVat", "OrderNo"]]

# Döp om kolumnerna till samma som vi kommer ha på datasetet data för att vi sedan ska kunna joina dem

data2.columns = ["Konto", "B", "C", "D", "E","Belopp", "Referens"]

# Gå nu tillbaka till datasetet data och skapa kolumnera data["Konto"] och data["Referens"]
# utan värde så att vi kan fylla på dem

data["Konto"] = None

data["Referens"] = None

# Ge konto-kolumnen ett utgångsvärde på 2976, eftersom det är vår vanligaste bokning off-season

data["Konto"] = "2976"

# Använd loc str contains  i kolumnen data['ProductName'] för att ge kolumnen data["Referens"] rätt referens

data.loc[data['ProductName'].str.contains('Guldkortet'), 'Referens'] = '3056/110'
data.loc[data['ProductName'].str.contains('Gröna Kortet'), 'Referens'] = '3018/110'
data.loc[data['ProductName'].str.contains('Åkband'), 'Referens'] = '3051/110'
data.loc[data['ProductName'].str.contains('Paketentré'), 'Referens'] = '3001/110'
data.loc[data['ProductName'].str.contains('Entré'), 'Referens'] = '3001/110'
data.loc[data['ProductName'].str.contains('House'), 'Referens'] = '3059/110'
data.loc[data['ProductName'].str.contains('Rab'), 'Referens'] = '3011/110'
data.loc[data['ProductName'].str.contains('Stora'), 'Referens'] = 'Stora Gruppaketet'

# Gör samma sak så att vi får rätt konto

data.loc[data['ProductName'].str.contains('House'), 'Konto'] = "2978"
data.loc[data['ProductName'].str.contains('Present'), 'Konto'] = "2894"
data.loc[data['ProductName'].str.contains('Stora'), 'Konto'] = "1915"
data.loc[data['Info'].str.contains('Hämta'), 'Konto'] = "2813"

# Skapa nya kolumner för moms - ett för 6% moms och ett för 25% moms

data["2610"] = None
data["2630"] = None

# Skapa en ny kolumn som matchar belopp-raden i data2

data["Belopp"] = None

# Fyll upp kolumnerna 2610 och 2630 efter vilken typ av intäkt det är

data.loc[data['Konto'].str.contains('2976'), '2610'] = data["Moms"]
data.loc[data['Konto'].str.contains('2978'), '2630'] = data["Moms"]

# Fyll upp kolumnen data['Belopp'] med ink- alternativt ex-moms priset beroende på vilket konto det har

data.loc[data['Konto'].str.contains('2976'), 'Belopp'] = data["PriceExcVat"]
data.loc[data['Konto'].str.contains('2978'), 'Belopp'] = data["PriceExcVat"]
data.loc[data['Konto'].str.contains('2813'), 'Belopp'] = data["PriceIncVat"]
data.loc[data['Konto'].str.contains('1915'), 'Belopp'] = data["PriceIncVat"]
data.loc[data['Konto'].str.contains('2894'), 'Belopp'] = data["PriceIncVat"]

# Summera momsen för både 2610 och 2630 i variablerna Moms_25 resp Moms_6

Moms_25 = data["2610"].sum()
Moms_6 = data["2630"].sum()

# Skapa två nya rader där värdet på variablerna läggs in (antalet kolumner måste stämma överens med
# resten av datasetet) samt där vi skriver in vilket konto som värdena ska ha

data.loc[-1] = [None, None, None, None, None, None, "2610", None, None, None, Moms_25]
data.loc[-2] = [None, None, None, None, None, None, "2630", None, None, None, Moms_6]

# Skapa fyra nya kolumner för att CSV-filen ska bli korrekt

data["B"] = None
data["C"]= None
data["D"] = None
data["E"] = None

# Skapa en identisk struktur som data2, dvs. att kolumnerna heter likadant

data = data[["Konto", "B", "C", "D", "E", "Belopp", "Referens"]]

# Skapa en variabel som heter frames, som innehåller både data och data2

frames = [data, data2]

# Skapa sedan ett nytt dataset som heter data3 genom att sammanföra data + data2

data3 = pd.concat(frames)

# Ta bort alla rader där kolumnen "Belopp" är 0 (för att slippa massa onödiga rader)

data3 = data3[data3.Belopp != 0]

# Eftersom en del belopp har decimaler gör vi om den kolumnen till Floats
# Vi gör även om kolumnen data3['Konto'] till int, så att vi kan läsa in den som CSV

data3["Belopp"] = data3["Belopp"].astype(float).replace('\.o', '', regex = True)

data3["Konto"] = data3["Konto"].astype(int).replace('\.o', '', regex = True)

# Spara ner csv-filen i samma map som vi använde för att öppna excel-filen, dvs. C:\Users\seosob02\

data3.to_csv('Återbetalning.csv', index=False, header=False, sep = ";") 